In [112]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.optimize import minimize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [114]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
1,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
2,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
3,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
4,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
5,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
6,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
7,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
8,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
9,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [115]:
#Scale for better intepretation

data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [116]:
# Creating dummy for china
data['China'] = (data['Country'] == 'CN').astype(int)

In [117]:
data = data.sort_values(['Year', 'ID']).reset_index(drop=True)

# Outside share

In [118]:
data.loc[data['Year'] == 2013, 'Market_share'] = data.loc[data['Year'] == 2013, 'Sales'] / 180632
data.loc[data['Year'] == 2014, 'Market_share'] = data.loc[data['Year'] == 2014, 'Sales'] / 188406
data.loc[data['Year'] == 2015, 'Market_share'] = data.loc[data['Year'] == 2015, 'Sales'] / 206653
data.loc[data['Year'] == 2016, 'Market_share'] = data.loc[data['Year'] == 2016, 'Sales'] / 222471
data.loc[data['Year'] == 2017, 'Market_share'] = data.loc[data['Year'] == 2017, 'Sales'] / 221471
data.loc[data['Year'] == 2018, 'Market_share'] = data.loc[data['Year'] == 2018, 'Sales'] / 252328
data.loc[data['Year'] == 2019, 'Market_share'] = data.loc[data['Year'] == 2019, 'Sales'] / 258727
data.loc[data['Year'] == 2020, 'Market_share'] = data.loc[data['Year'] == 2020, 'Sales'] / 230060
data.loc[data['Year'] == 2021, 'Market_share'] = data.loc[data['Year'] == 2021, 'Sales'] / 222210
data.loc[data['Year'] == 2022, 'Market_share'] = data.loc[data['Year'] == 2022, 'Sales'] / 181030
data.loc[data['Year'] == 2023, 'Market_share'] = data.loc[data['Year'] == 2023, 'Sales'] / 203690

In [119]:
data['outside_share'] = 1 - data.groupby('Year')['Market_share'].transform('sum')
data[['Market_share', 'outside_share']].describe()

,Market_share,outside_share
count,334.000000,334.000000
mean,0.002128,0.840911
std,0.005618,0.108763
min,0.000004,0.696180
25%,0.000123,0.696180
50%,0.000715,0.830851
75%,0.002161,0.938460
max,0.088139,0.997525


# Instruments

In [120]:
fun.GH(data, 'Range', 0.5)
fun.GH(data, 'HP', 0.5)
fun.GH(data, 'Chargetime', 0.5)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,outside_share,Range_GH,HP_GH,Chargetime_GH
0,10,2013,0.000006,BMW,I3,29.5,25.00000,16.7,18,Hatchback,B,DE,1,1.0,0,0.997525,110.1,83.6,237
1,132,2013,0.001168,Nissan,Leaf,32.8,25.36900,14.7,43,Hatchback,C,JP,211,1.0,0,0.997525,99.2,67.5,164
2,158,2013,0.000509,Renault,Zoe,36.5,17.31500,13.4,56,Hatchback,B,FR,92,1.0,0,0.997525,128.7,67.5,139
3,167,2013,0.000006,Smart,Fortwo,12.7,21.04575,8.0,60,Hatchback,A,DE,1,1.0,0,0.997525,185.3,84.2,139
4,173,2013,0.000620,Tesla,Model S,60.9,71.95631,67.5,30,Liftback,F,US,112,1.0,0,0.997525,137.1,60.9,225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,187,2023,0.000388,Volkswagen,up!,25.6,17.48530,8.1,48,Hatchback,A,DE,79,1.0,0,0.696180,3737.4,2269.0,2733
330,188,2023,0.002538,Volvo,C40,46.6,43.06649,40.2,28,SUV,C,SE,517,1.0,0,0.696180,2212.4,1978.6,1813
331,189,2023,0.000020,Volvo,EX30,47.5,36.82450,26.8,28,SUV,B,SE,4,1.0,0,0.696180,2189.4,1777.6,1813
332,190,2023,0.009647,Volvo,XC40,45.7,43.92666,40.2,28,SUV,C,SE,1965,1.0,0,0.696180,2206.7,1978.6,1813


# Pure Logit

In [121]:
formula = 'np.log(Market_share/outside_share) ~ 1 + [Price ~ Range_GH + HP_GH + Chargetime_GH] + Range + HP + Chargetime + China '
IV = IV2SLS.from_formula(formula, data).fit(cov_type='robust')
IV.first_stage

,Price
R-squared,0.6105
Partial R-squared,0.0919
Shea's R-squared,0.0919
Partial F-statistic,22.829
P-value (Partial F-stat),4.383e-05
Partial F-stat Distn,chi2(3)
==========================,===========
Intercept,1.7422
,(0.2392)
Range,0.4165


In [122]:
IV.summary#.tables[1]

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                                   IV-2SLS Estimation Summary                                   
================================================================================================
Dep. Variable:     np.log(Market_share / outside_share)   R-squared:                      0.0795
Estimator:                                      IV-2SLS   Adj. R-squared:                 0.0654
No. Observations:                                   334   F-statistic:                    145.01
Date:                                  Tue, Jun 04 2024   P-value (F-stat)                0.0000
Time:                                          18:27:24   Distribution:                  chi2(5)
Cov. Estimator:                                  robust                                         
                                                                                                
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -8.0044     0.7315    -10.942     0.0000     -9.4382     -6.5707
Range          0.1312     0.0148     8.8514     0.0000      0.1021      0.1603
HP             0.0247     0.0255     0.9706     0.3317     -0.0252      0.0747
Chargetime    -0.0519     0.0113    -4.5869     0.0000     -0.0740     -0.0297
China         -1.4436     0.3839    -3.7599     0.0002     -2.1961     -0.6911
Price         -0.0712     0.0202    -3.5224     0.0004     -0.1108     -0.0316
==============================================================================

Endogenous: Price
Instruments: Range_GH, HP_GH, Chargetime_GH
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Willingness to pay

In [123]:
alpha = IV.params[-1] # Price coefficient
beta = IV.params[:-1]
beta_alpha_ratio = [b / -alpha for b in beta[1:]] #Willingness to pay (excluding constant)

for i in range(len(IV.params.index[1:-1])): #[1:-1] to exclude constant and price
    print('W2P:', IV.params.index[1:-1][i], beta_alpha_ratio[i])

W2P: Range 1.8432176840440782
W2P: HP 0.34736739104455305
W2P: Chargetime -0.7284676591201457
W2P: China -20.28051213973809


NOTE: English delimiter\
8,102 DKK for every 10 increase in Range\
9,217 DKK for every 10 increase in HP\
-2,773 DKK for every 10 increase in Chargetime\
-220,368 DKK for Chinese cars OR -22,036 DKK for Chinese cars???

# Substitution patterns

In [124]:
# IMPORTANT: The data must be sorted by year and ID before running the function
#             Because the CCPs returned by the function are sorted by year and ID
logit_data = data.sort_values(['Year', 'ID']).reset_index(drop=True)
X = logit_data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]
p_j = logit_data['Price']
logit_data['CCP'] = fun.ccp(alpha, beta, data, X)

In [125]:
probability_ratio = fun.probability_ratio(logit_data, 2023)
probability_ratio

Model,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,...,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
Model,,,,,,,,,,,,,,,,,,,,,
U5,1.0,1.797495,1.512492,15.378865,0.66544,1.109559,0.644494,0.290039,0.414738,594.991883,...,0.249436,0.112712,0.091554,0.05491,1.043546,2.293491,0.144426,0.11463,0.172789,2.20817
U6,0.55633,1.0,0.841444,8.555722,0.370204,0.617281,0.358551,0.161357,0.230731,331.011759,...,0.138769,0.062705,0.050934,0.030548,0.580556,1.275938,0.080348,0.063772,0.096128,1.228471
e-tron,0.661161,1.188433,1.0,10.167899,0.439962,0.733597,0.426114,0.191762,0.274208,393.385169,...,0.164917,0.074521,0.060532,0.036304,0.689951,1.516366,0.095489,0.075789,0.114241,1.459955
e-tron GT,0.065024,0.116881,0.098349,1.0,0.04327,0.072148,0.041908,0.01886,0.026968,38.688934,...,0.016219,0.007329,0.005953,0.00357,0.067856,0.149133,0.009391,0.007454,0.011236,0.143585
Q4 e-tron,1.502766,2.701214,2.272922,23.110838,1.0,1.667407,0.968524,0.435861,0.623254,894.133678,...,0.374844,0.16938,0.137585,0.082516,1.568205,3.446581,0.217038,0.172261,0.259662,3.318364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,0.436017,0.783737,0.659471,6.705439,0.290143,0.483786,0.28101,0.126462,0.180833,259.42628,...,0.108758,0.049144,0.039919,0.023941,0.455003,1.0,0.062972,0.04998,0.075339,0.962799
C40,6.923968,12.445796,10.472446,106.482768,4.607482,7.68255,4.462455,2.00822,2.871633,4119.704833,...,1.727089,0.780416,0.63392,0.380192,7.225477,15.88006,1.0,0.793691,1.196387,15.289301
EX30,8.723757,15.680908,13.194613,134.161485,5.805133,9.679522,5.622408,2.530229,3.618074,5190.56489,...,2.176022,0.983274,0.798699,0.479018,9.10364,20.007861,1.259936,1.0,1.507372,19.263543


In [126]:
logit_data['Model_year'] = logit_data['Model'] + '_' + logit_data['Year'].astype(str)
marginal_effects = fun.marginal_effects(logit_data, IV)
marginal_effects

,Intercept,Range,HP,Chargetime,China,Price
Model_year,,,,,,
I3_2013,-1.077563,0.017662,0.003329,-0.00698,-0.194334,-0.009582
Leaf_2013,-0.470314,0.007709,0.001453,-0.003047,-0.084819,-0.004182
Zoe_2013,-0.650167,0.010657,0.002008,-0.004212,-0.117255,-0.005782
Fortwo_2013,-0.017107,0.00028,0.000053,-0.000111,-0.003085,-0.000152
Model S_2013,-1.802674,0.029547,0.005568,-0.011678,-0.325104,-0.01603
...,...,...,...,...,...,...
up!_2023,-0.009527,0.000156,0.000029,-0.000062,-0.001718,-0.000085
C40_2023,-0.148604,0.002436,0.000459,-0.000963,-0.0268,-0.001321
EX30_2023,-0.186293,0.003054,0.000575,-0.001207,-0.033597,-0.001657


In [127]:
marginal_effects[IV.params.index].mean()

Intercept    -0.204855
Range         0.003358
HP            0.000633
Chargetime   -0.001327
China        -0.036945
Price        -0.001822
dtype: float64

In [128]:
elasticity = fun.elasticity(logit_data, IV)
elasticity

,Intercept,Range,HP,Chargetime,China,Price
Model_year,,,,,,
I3_2013,-6.721119,3.249872,0.346715,-0.783701,-0.0,-1.494196
Leaf_2013,-7.502661,4.033591,0.340681,-2.089874,-0.0,-1.692562
Zoe_2013,-7.290604,4.361733,0.301775,-2.64477,-0.0,-1.122566
Fortwo_2013,-7.987286,1.662668,0.197381,-3.104465,-0.0,-1.494823
Model S_2013,-2.74192,2.736999,0.571707,-0.532859,-0.0,-1.754485
...,...,...,...,...,...,...
up!_2023,-7.99489,3.354711,0.200038,-2.485937,-0.0,-1.243116
C40_2023,-7.852958,5.998212,0.975157,-1.424386,-0.0,-3.007452
EX30_2023,-7.813585,6.083403,0.646845,-1.417244,-0.0,-2.558664


In [129]:
elasticity[IV.params.index].mean()

Intercept    -7.740810
Range         4.981253
HP            0.597185
Chargetime   -1.716720
China        -0.168143
Price        -3.232126
dtype: float64

# Analysis on subsample

### The subsample consists of the 2023 market with:
The 5 highest market share models: Model Y, Model 3, Enyag iV, ID.4 and Q4 e-tron\
The 5 highest market share chinese models: 4, Euniq6, Atto 3, Marvel R and Dolphin\
The highest ccp model: Ocean\
The highest ccp chinese model: Seal\
The highest ccp korean model (so we have models outside EU, US and china): Ioniq 6\
Polestar 2 as a random interest


In [130]:
analysis_data = logit_data[logit_data['Year']==2023].copy()
analysis_data = analysis_data[analysis_data['Model'].isin(['Model 3', 'Model Y', 'ID.4', 'Enyaq iV', 'Ocean' , '2', 'Ioniq 6', 'Q4 e-tron'
                                                           , '4', 'Euniq6', 'Atto 3', 'Marvel R', 'Dolphin', 'Seal'])]
analysis_data.reset_index(drop=True, inplace=True)
analysis_data


,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,Country,Sales,Intercept,China,outside_share,Range_GH,HP_GH,Chargetime_GH,CCP,Model_year
0,5,2023,0.011532,Audi,Q4 e-tron,49.6,65.855307,28.1,28,SUV,...,DE,2349,1.0,0,0.69618,2315.6,1725.6,1813,0.004107,Q4 e-tron_2023
1,17,2023,0.001090,BYD,Atto 3,42.0,31.000000,20.1,37,SUV,...,CN,222,1.0,1,0.69618,2569.2,1501.0,2195,0.002200,Atto 3_2023
2,18,2023,0.000997,BYD,Dolphin,42.7,23.323517,9.3,40,Hatchback,...,CN,203,1.0,1,0.69618,2605.7,2235.6,2006,0.002730,Dolphin_2023
3,20,2023,0.000157,BYD,Seal,54.0,32.062000,52.2,38,Sedan,...,CN,32,1.0,1,0.69618,2900.5,2103.5,2184,0.020681,Seal_2023
4,49,2023,0.001424,Fisker,Ocean,70.0,60.237920,56.3,35,SUV,...,DK,290,1.0,0,0.69618,4079.7,2167.7,2066,0.124379,Ocean_2023
5,69,2023,0.000972,Hyundai,Ioniq 6,56.7,43.629166,32.0,16,Sedan,...,KR,198,1.0,0,0.69618,3065.5,1958.7,3157,0.104057,Ioniq 6_2023
6,100,2023,0.001213,Maxus,Euniq6,35.4,37.950676,17.4,45,MPV,...,CN,247,1.0,1,0.69618,3271.4,1752.3,2407,0.000349,Euniq6_2023
7,117,2023,0.003643,MG,4,42.7,26.589191,24.1,41,Hatchback,...,CN,742,1.0,1,0.69618,2605.7,1700.9,2054,0.002962,4_2023
8,119,2023,0.001021,MG,Marvel R,38.8,30.235650,17.7,38,SUV,...,CN,208,1.0,1,0.69618,3041.5,1752.3,2184,0.001366,Marvel R_2023
9,149,2023,0.009927,Polestar,2,51.3,38.989190,41.5,28,Liftback,...,SE,2022,1.0,0,0.69618,2459.4,1978.6,1813,0.048395,2_2023


In [131]:
cross_elasticity = fun.cross_elasticity(analysis_data, IV, 'Price')
cross_elasticity 

Model_year,Q4 e-tron_2023,Atto 3_2023,Dolphin_2023,Seal_2023,Ocean_2023,Ioniq 6_2023,Euniq6_2023,4_2023,Marvel R_2023,2_2023,Enyaq iV_2023,Model 3_2023,Model Y_2023,ID.4_2023
Model_year,,,,,,,,,,,,,,
Q4 e-tron_2023,0.01925,0.00485,0.00453,0.0472,0.5333,0.32315,0.00094,0.00561,0.00294,0.13431,0.06336,0.18769,0.15036,0.07252
Atto 3_2023,0.01925,0.00485,0.00453,0.0472,0.5333,0.32315,0.00094,0.00561,0.00294,0.13431,0.06336,0.18769,0.15036,0.07252
Dolphin_2023,0.01925,0.00485,0.00453,0.0472,0.5333,0.32315,0.00094,0.00561,0.00294,0.13431,0.06336,0.18769,0.15036,0.07252
Seal_2023,0.01925,0.00485,0.00453,0.0472,0.5333,0.32315,0.00094,0.00561,0.00294,0.13431,0.06336,0.18769,0.15036,0.07252
Ocean_2023,0.01925,0.00485,0.00453,0.0472,0.5333,0.32315,0.00094,0.00561,0.00294,0.13431,0.06336,0.18769,0.15036,0.07252
Ioniq 6_2023,0.01925,0.00485,0.00453,0.0472,0.5333,0.32315,0.00094,0.00561,0.00294,0.13431,0.06336,0.18769,0.15036,0.07252
Euniq6_2023,0.01925,0.00485,0.00453,0.0472,0.5333,0.32315,0.00094,0.00561,0.00294,0.13431,0.06336,0.18769,0.15036,0.07252
4_2023,0.01925,0.00485,0.00453,0.0472,0.5333,0.32315,0.00094,0.00561,0.00294,0.13431,0.06336,0.18769,0.15036,0.07252
Marvel R_2023,0.01925,0.00485,0.00453,0.0472,0.5333,0.32315,0.00094,0.00561,0.00294,0.13431,0.06336,0.18769,0.15036,0.07252


In [132]:
cross_elasticity_1 = fun.cross_elasticity_1(analysis_data, IV)
cross_elasticity_1[50:100]

/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:195: PerformanceWarning: indexing past lexsort depth may impact performance.
  cross_elasticity_table.loc[(model_labels[i], model_labels[j], X.columns[k]), 'Cross_Elasticity'] = -coefficients[k] * X.iloc[j, k] * ccp.iloc[j]


Cross_Elasticity
Model_year     Model_year                                
Q4 e-tron_2023 Marvel R_2023  HP                -0.000598
                              Chargetime         0.002692
                              China              0.001972
                              Price               0.00294
               2_2023         Intercept          0.387376
                              Range             -0.325726
                              HP                -0.049659
                              Chargetime         0.070263
                              China                   0.0
                              Price              0.134308
               Enyaq iV_2023  Intercept          0.164954
                              Range             -0.134106
                              HP                -0.014318
                              Chargetime          0.02992
                              China                   0.0
                              Price              0.063361
               Model 3_2023   Intercept          0.587951
                              Range             -0.522328
                              HP                 -0.05049
                              Chargetime         0.095218
                              China                   0.0
                              Price              0.187694
               Model Y_2023   Intercept          0.416933
                              Range             -0.345795
                              HP                -0.065168
                              Chargetime         0.072923
                              China                   0.0
                              Price              0.150363
               ID.4_2023      Intercept           0.19409
                              Range             -0.159702
                              HP                -0.016847
                              Chargetime         0.035204
                              China                   0.0
                              Price              0.072517
Atto 3_2023    Q4 e-tron_2023 Intercept          0.032875
                              Range             -0.026727
                              HP                -0.002854
                              Chargetime         0.005963
                              China                   0.0
                              Price              0.019252
               Atto 3_2023    Intercept          0.017611
                              Range             -0.012124
                              HP                -0.001093
                              Chargetime         0.004221
                              China              0.003176
                              Price              0.004855
               Dolphin_2023   Intercept           0.02185
                              Range             -0.015293
                              HP                -0.000628
                              Chargetime         0.005662

# Cost

In [133]:
#logit_data['Cost'] = fun.cost(logit_data, alpha)
#logit_data = fun.cost_firm_OLD(logit_data, alpha)
logit_data = fun.cost_firm(logit_data, alpha)
logit_data = fun.markup(logit_data)
logit_data[logit_data['Manufacturer']=='Tesla']

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,Intercept,China,outside_share,Range_GH,HP_GH,Chargetime_GH,CCP,Model_year,firm_cost,markup%
4,173,2013,0.000620,Tesla,Model S,60.9,71.95631,67.5,30,Liftback,...,1.0,0,0.997525,137.1,60.9,225,0.657450,Model S_2013,62.719837,14.726557
9,173,2014,0.002452,Tesla,Model S,60.9,66.39721,67.5,30,Liftback,...,1.0,0,0.992118,147.4,66.3,201,0.749505,Model S_2014,55.867458,18.847739
16,173,2015,0.013186,Tesla,Model S,60.9,71.82500,67.5,30,Liftback,...,1.0,0,0.980925,192.4,86.4,245,0.550212,Model S_2015,64.095099,12.060050
23,173,2016,0.000351,Tesla,Model S,60.9,95.12510,67.5,30,Liftback,...,1.0,0,0.994651,245.3,86.4,245,0.193656,Model S_2016,92.080730,3.306197
24,174,2016,0.000441,Tesla,Model X,52.9,110.28690,67.5,30,SUV,...,1.0,0,0.994651,253.3,86.4,245,0.023041,Model X_2016,107.242530,2.838771
32,173,2017,0.000208,Tesla,Model S,60.9,96.90777,67.5,30,Liftback,...,1.0,0,0.997065,276.3,98.2,264,0.157594,Model S_2017,94.493603,2.554847
33,174,2017,0.000235,Tesla,Model X,52.9,115.92930,67.5,30,SUV,...,1.0,0,0.997065,284.3,98.2,264,0.014246,Model X_2017,113.515133,2.126736
43,173,2018,0.000186,Tesla,Model S,60.9,93.49610,67.5,30,Liftback,...,1.0,0,0.994186,367.1,159.0,349,0.145065,Model S_2018,91.287020,2.419928
44,174,2018,0.000190,Tesla,Model X,52.9,113.55900,67.5,30,SUV,...,1.0,0,0.994186,375.1,159.0,349,0.012177,Model X_2018,111.349920,1.983908
58,172,2019,0.009427,Tesla,Model 3,54.2,43.77533,27.8,25,Sedan,...,1.0,0,0.978881,511.3,340.9,502,0.422856,Model 3_2019,36.256773,20.736971


In [134]:
cost_side = sm.ols('np.log(firm_cost) ~ Range + HP + Chargetime', logit_data).fit(cov_type='HC3')
cost_side.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(firm_cost)   R-squared:                       0.651
Model:                            OLS   Adj. R-squared:                  0.647
Method:                 Least Squares   F-statistic:                     166.0
Date:                Tue, 04 Jun 2024   Prob (F-statistic):           1.39e-65
Time:                        18:27:30   Log-Likelihood:                -85.759
No. Observations:                 334   AIC:                             179.5
Df Residuals:                     330   BIC:                             194.8
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.1122      0.096     32.282      0.000       2.923       3.301
Range          0.0084      0.003      3.360      0.001       0.004       0.013
HP             0.0199      0.002     13.045      0.000       0.017       0.023
Chargetime    -0.0080      0.002     -4.831      0.000      -0.011      -0.005
==============================================================================
Omnibus:                       38.354   Durbin-Watson:                   1.530
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               51.197
Skew:                           0.804   Prob(JB):                     7.63e-12
Kurtosis:                       4.046   Cond. No.                         333.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Nash Equilibrium on subsample

In [135]:
NE_data = logit_data[logit_data['Year']==2023].copy()
NE_data.reset_index(drop=True, inplace=True)
X_ne = NE_data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]

NE_analysis_data = NE_data[NE_data['Model'].isin(['Model 3', 'Model Y', 'ID.4', 'Enyaq iV', 'Ocean' , '2', 'Ioniq 6', 'Q4 e-tron'
                                                           , '4', 'Euniq6', 'Atto 3', 'Marvel R', 'Dolphin', 'Seal'])]
NE_analysis_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,Intercept,China,outside_share,Range_GH,HP_GH,Chargetime_GH,CCP,Model_year,firm_cost,markup%
4,5,2023,0.011532,Audi,Q4 e-tron,49.6,65.855307,28.1,28,SUV,...,1.0,0,0.69618,2315.6,1725.6,1813,0.004107,Q4 e-tron_2023,65.735119,0.182837
13,17,2023,0.001090,BYD,Atto 3,42.0,31.000000,20.1,37,SUV,...,1.0,1,0.69618,2569.2,1501.0,2195,0.002200,Atto 3_2023,30.610983,1.270840
14,18,2023,0.000997,BYD,Dolphin,42.7,23.323517,9.3,40,Hatchback,...,1.0,1,0.69618,2605.7,2235.6,2006,0.002730,Dolphin_2023,22.934500,1.696208
16,20,2023,0.000157,BYD,Seal,54.0,32.062000,52.2,38,Sedan,...,1.0,1,0.69618,2900.5,2103.5,2184,0.020681,Seal_2023,31.672983,1.228229
26,49,2023,0.001424,Fisker,Ocean,70.0,60.237920,56.3,35,SUV,...,1.0,0,0.69618,4079.7,2167.7,2066,0.124379,Ocean_2023,58.490532,2.987471
32,69,2023,0.000972,Hyundai,Ioniq 6,56.7,43.629166,32.0,16,Sedan,...,1.0,0,0.69618,3065.5,1958.7,3157,0.104057,Ioniq 6_2023,41.620779,4.825443
43,100,2023,0.001213,Maxus,Euniq6,35.4,37.950676,17.4,45,MPV,...,1.0,1,0.69618,3271.4,1752.3,2407,0.000349,Euniq6_2023,37.910989,0.104684
54,117,2023,0.003643,MG,4,42.7,26.589191,24.1,41,Hatchback,...,1.0,1,0.69618,2605.7,1700.9,2054,0.002962,4_2023,26.493785,0.360108
56,119,2023,0.001021,MG,Marvel R,38.8,30.235650,17.7,38,SUV,...,1.0,1,0.69618,3041.5,1752.3,2184,0.001366,Marvel R_2023,30.140244,0.316541
74,149,2023,0.009927,Polestar,2,51.3,38.989190,41.5,28,Liftback,...,1.0,0,0.69618,2459.4,1978.6,1813,0.048395,2_2023,38.309288,1.774770


In [136]:
car1 = 16
car2 = 85 # Equilibrium bilen

p1_data = NE_data.loc[car1, 'Price'] 
p2_data = NE_data.loc[car2, 'Price'] 

c1 = NE_data.loc[car1, 'firm_cost']
c2 = NE_data.loc[car2, 'firm_cost']

def market_shares(p1, p2): 
    x2 = NE_data.copy() # copy to avoid writing to the original data 
    fun.set_car_price(x2, p1, car1) # Set new price for car 1
    fun.set_car_price(x2, p2, car2) # set new price for car 2
    ccp = fun.ccp(alpha, beta, x2, X_ne) # compute the CCPs for the new prices
    s1 = ccp[car1] 
    s2 = ccp[car2] 
    return s1, s2

def profit(p, s, c): 
    return s * (p - c)

def profit1(p1, p2): 
    s1, s2 = market_shares(p1, p2)
    pi1 = profit(p1, s1, c1)
    return pi1

def profit2(p2, p1): 
    s1, s2 = market_shares(p1, p2)
    pi2 = profit(p2, s2, c2)
    return pi2

In [137]:
p2 = p2_data # initial guess

for i in range(10):
    f = lambda p: -profit1(p1=p, p2=p2)
    res = minimize(f, x0=p2)
    p1 = res['x'][0]
    print(f'BR1(p2={p2:6.4f}) = {p1:6.4f}')

    f = lambda p: -profit2(p1=p1, p2=p)
    res = minimize(f, x0=p2)
    p2 = res['x'][0]
    print(f'BR2(p1={p1:6.4f}) = {p2:6.4f}')

BR1(p2=40.5554) = 45.8157
BR2(p1=45.8157) = 53.1335
BR1(p2=53.1335) = 45.8372
BR2(p1=45.8372) = 53.1335
BR1(p2=53.1335) = 45.8372
BR2(p1=45.8372) = 53.1335
BR1(p2=53.1335) = 45.8372
BR2(p1=45.8372) = 53.1335
BR1(p2=53.1335) = 45.8372
BR2(p1=45.8372) = 53.1335
BR1(p2=53.1335) = 45.8372
BR2(p1=45.8372) = 53.1335
BR1(p2=53.1335) = 45.8372
BR2(p1=45.8372) = 53.1335
BR1(p2=53.1335) = 45.8372
BR2(p1=45.8372) = 53.1335
BR1(p2=53.1335) = 45.8372
BR2(p1=45.8372) = 53.1335
BR1(p2=53.1335) = 45.8372
BR2(p1=45.8372) = 53.1335
